In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time
import os

from bs4 import BeautifulSoup
import re
import pandas as pd

import requests
import os
import urllib
from urllib.parse import urlparse

import subprocess


In [2]:
#settings

IMPLICIT_DELAY = 10
EXPLICIT_DELAY = 10
TAKE_SCREENSHOT = False

In [3]:
# default settings for this Udacity program

download_root_folder = ""
current_read_write_path = ""

# hard-coded entry url for the program
root_address = 'https://learn.udacity.com/nanodegrees/nd880/parts/cd0569'

In [4]:
#setup option for chrome profile
chrome_options = Options()

chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
## the following option is to solve the error if using the above cause a problem
# chrome_options.add_argument("--remote-debugging-port=9222") 

prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)
if TAKE_SCREENSHOT:
    chrome_options.add_argument('--headless')

#start web driver
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(IMPLICIT_DELAY)
wait = WebDriverWait(driver, EXPLICIT_DELAY)

In [5]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException

def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

def sync_get_elements_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_all_elements_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_elements("xpath",xpath)

In [6]:
def take_screenshot(url, screenshot_file_path):
    # start a new web driver headlessly to take screenshots

    #setup option for chrome profile
    chrome_options = Options()

    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    ## the following option is to solve the error if using the above cause a problem
    # chrome_options.add_argument("--remote-debugging-port=9222") 

    prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", prefs)

    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--start-maximized")

    #start web driver
    driver_t = webdriver.Chrome(options=chrome_options)
    driver_t.implicitly_wait(IMPLICIT_DELAY)
    wait_t = WebDriverWait(driver_t, EXPLICIT_DELAY)


    driver_t.get(url)
    print("start headless temp chrome driver. get {}".format(url))
    
    # sign in. when chrome_options user-data-dir is turned on, no need to sign in thus just need to wait for timeout

    signin_btn = sync_get_element_by_xpath(driver_t, wait_t, "//div[text()='Sign in']")
    if signin_btn is not None:
        signin_btn.click()
        print("switch to sign-in tab")

        email = sync_get_element_by_xpath(driver_t, wait_t, "//input[@id='email']")
        password = sync_get_element_by_xpath(driver_t, wait_t, "//input[@id='revealable-password']")

        if email is not None:
            email.send_keys("huohsien@gmail.com")
        if password is not None:
            password.send_keys("Huo18941256")

        signin_btn = sync_get_element_by_xpath(driver_t, wait_t, "//div[@data-testid='signin-form']//span[text()='Sign in']/..")
        
        signin_btn.click()
        print("sign in")

        # close chaptgpt help side panel

        btn = sync_get_element_by_xpath(driver_t, wait_t, "//button[@class='chakra-button css-twu8br']")
        print("try to close chat panel")
        if btn is not None:
            try:
                btn.click()
            except ElementNotInteractableException:
                print("chat panel was closed")
        print("start sleeping for 5 secs")
        time.sleep(5)

    height = driver_t.execute_script('var pageHeight = 0;function findHighestNode(nodesList) {for (var i = nodesList.length - 1; i >= 0; i--) {if (nodesList[i].scrollHeight && nodesList[i].clientHeight) {var elHeight = Math.max(nodesList[i].scrollHeight, nodesList[i].clientHeight);pageHeight = Math.max(elHeight, pageHeight);}if (nodesList[i].childNodes.length) findHighestNode(nodesList[i].childNodes);}} findHighestNode(document.documentElement.childNodes);return pageHeight;')
    width  = driver_t.execute_script('return document.documentElement.scrollWidth')
    driver_t.set_window_size(width, height)  # the trick
    print("width: {} height: {}".format(width,height))
    
    print("start sleeping for 2 secs")
    time.sleep(2)
    driver_t.save_screenshot(screenshot_file_path)
    driver_t.quit()

In [7]:
# go to udacity
driver.get(root_address)

In [8]:
# # sign in. when chrome_options user-data-dir is turned on, no need to sign in thus just need to wait for timeout

# signin_btn = sync_get_element_by_xpath(driver, wait, "//div[text()='Sign in']")
# if signin_btn is not None:
#     signin_btn.click()

#     email = sync_get_element_by_xpath(driver, wait, "//input[@id='email']")
#     password = sync_get_element_by_xpath(driver, wait, "//input[@id='revealable-password']")

#     if email is not None:
#         email.send_keys("huohsien@gmail.com")
#     if password is not None:
#         password.send_keys("Huo18941256")

#     signin_btn = sync_get_element_by_xpath(driver, wait, "//div[@data-testid='signin-form']//span[text()='Sign in']/..")
#     if signin_btn is not None:
#         signin_btn.click()


In [9]:
# get the name of the program and create the root folder for storing the scraped result
program_name = sync_get_element_by_xpath(driver, wait, "//h1[@class='chakra-heading css-10cnqhb']").get_attribute("innerHTML")
download_root_folder = program_name
current_read_write_path = download_root_folder
os.makedirs(current_read_write_path, exist_ok=False)

In [10]:
# get the two main part of this program. Hard coded!
course_content_blk1 = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panels css-8atqhb']/div[1]//div[@class='css-pk1kta']")[0]
course_content_blk2 = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panels css-8atqhb']/div[1]//div[@class='css-1vc4pt9']")[0] 

In [11]:
# create folder structure for storing the scraped data for the Udacity program
title_first_part = course_content_blk1.find_element("xpath", "./div/div/button/div/div[1]/div/div/h2").get_attribute("innerHTML")
title_second_part = course_content_blk2.find_element("xpath", "./div/div/button/div/div[1]/div/div/h2").get_attribute("innerHTML")

# keep reminding myself the current read/write folder
current_target_folder = current_read_write_path+"/1. "+title_first_part
os.makedirs(current_target_folder, exist_ok=False)
current_target_folder = current_read_write_path+"/2. "+title_second_part
os.makedirs(current_target_folder, exist_ok=False)

In [12]:
# get all the web elements of topics in the first part of the program
class_as = course_content_blk1.find_elements("xpath", "./div/div/div/div/a")

In [13]:
current_read_write_path = download_root_folder+"/1. "+title_first_part
topic_urls = []
class_as = course_content_blk1.find_elements("xpath", "./div/div/div/div/a")

# for idx in range(len(class_as)):
for idx in[0]:
    class_a = class_as[idx]
    class_title = class_a.find_element("xpath","./div/h3").get_attribute("innerHTML")
    current_target_folder = current_read_write_path+"/{}. ".format(idx)+class_title
    os.makedirs(current_target_folder, exist_ok=False)
    print("create folder: {}".format(current_target_folder))

    url = class_a.get_attribute("href")
    
    topic_urls.append(url)


create folder: AI for Trading/1. Quantitative Trading/0. Welcome to the Nanodegree Program


In [14]:
# take_screenshot(url,current_target_folder+"/screenshot.png")

In [15]:
for topic_url in topic_urls:
    print(topic_url)

https://learn.udacity.com/nanodegrees/nd880/parts/cd0569/lessons/ff736671-9e1a-4f97-b73d-a5211598c2ec


In [16]:
driver.get(topic_urls[0])

In [17]:
# close chaptgpt help side panel
btn = sync_get_element_by_xpath(driver, wait, "//button[@class='chakra-button css-twu8br']")
if btn is not None:
    try:
        btn.click()
    except ElementNotInteractableException:
        print("chatgpt panel was closed")

chatgpt panel was closed


In [18]:
def grab_all_youtube_links(content_html_str, download=False):

    content_soup = BeautifulSoup(content_html_str, 'html.parser')
    youtube_iframe_elms = content_soup.find_all('iframe', {'title': 'YouTube video player'})
    video_links = ''
    for idx, youtube_iframe_elm in enumerate(youtube_iframe_elms, start=1):
        video_link = youtube_iframe_elm['src']
        video_link = video_link + '#index={}_{}_{}_{}'.format(topic_name,lesson_name, part_name,idx)
        video_links = video_links + video_link + '\n\n'


    with open('all_youtube_video_links.txt','a') as fp:
        fp.writelines(video_links)

In [19]:
# e_header = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-4nu87b']")
# e_content = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-dmdipe']")

# header_height = int(e_header[0].get_attribute("scrollHeight"))
# content_height = int(e_content[0].get_attribute("scrollHeight"))

# full_page_height = content_height+header_height

# full_page_width = int(e_header[0].get_attribute("scrollWidth"))

In [20]:
if TAKE_SCREENSHOT:

    driver.set_window_size(full_page_width,full_page_height)

    el = driver.find_element(By.TAG_NAME, "body")
    el.screenshot("main.png")


In [21]:
# # get tab button web elements for different types of Curriculum: Core, Elective and Career
# course_tab_elms = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panel css-1qbr3jw']")

# tab_buttons = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tablist css-1cul31m']/button")
# for tab_button in tab_buttons:
#     print("Type: {}".format(tab_button.get_attribute("innerHTML")))


In [22]:
# take_screenshot("https://learn.udacity.com/nanodegrees/nd880/parts/cd0569/lessons/ff736671-9e1a-4f97-b73d-a5211598c2ec/concepts/ee2c9346-f9d4-4bef-8456-b54ddfeec2b5","main.png")

In [25]:
def test_fullpage_screenshot(url, screenshot_file_path):

    # start a new web driver headlessly to take screenshots

    #setup option for chrome profile
    chrome_options = Options()

    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    ## the following option is to solve the error if using the above cause a problem
    # chrome_options.add_argument("--remote-debugging-port=9222") 

    prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", prefs)

    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--start-maximized")

    #start web driver
    driver_t = webdriver.Chrome(options=chrome_options)
    driver_t.implicitly_wait(IMPLICIT_DELAY)
    wait_t = WebDriverWait(driver_t, EXPLICIT_DELAY)


    driver_t.get(url)
    print("start headless temp chrome driver. get {}".format(url))
    
    # sign in. when chrome_options user-data-dir is turned on, no need to sign in thus just need to wait for timeout

    signin_btn = sync_get_element_by_xpath(driver_t, wait_t, "//div[text()='Sign in']")
    if signin_btn is not None:
        signin_btn.click()

        email = sync_get_element_by_xpath(driver_t, wait_t, "//input[@id='email']")
        password = sync_get_element_by_xpath(driver_t, wait_t, "//input[@id='revealable-password']")

        if email is not None:
            email.send_keys("huohsien@gmail.com")
        if password is not None:
            password.send_keys("Huo18941256")

        signin_btn = sync_get_element_by_xpath(driver_t, wait_t, "//div[@data-testid='signin-form']//span[text()='Sign in']/..")

        signin_btn.click()

        # close chaptgpt help side panel

        btn = sync_get_element_by_xpath(driver_t, wait_t, "//button[@class='chakra-button css-twu8br']")
        if btn is not None:
            try:
                btn.click()
            except ElementNotInteractableException:
                print("chatgpt panel was closed")

        time.sleep(5)

    height = driver_t.execute_script('var pageHeight = 0;function findHighestNode(nodesList) {for (var i = nodesList.length - 1; i >= 0; i--) {if (nodesList[i].scrollHeight && nodesList[i].clientHeight) {var elHeight = Math.max(nodesList[i].scrollHeight, nodesList[i].clientHeight);pageHeight = Math.max(elHeight, pageHeight);}if (nodesList[i].childNodes.length) findHighestNode(nodesList[i].childNodes);}} findHighestNode(document.documentElement.childNodes);return pageHeight;')
    width  = driver_t.execute_script('return document.documentElement.scrollWidth')
    driver_t.set_window_size(width, height)  # the trick
    print("width: {} height: {}".format(width,height))
    time.sleep(2)
    driver_t.save_screenshot(screenshot_file_path)
    driver_t.quit()
# test_fullpage_screenshot("https://learn.udacity.com/nanodegrees/nd880/parts/cd0569/lessons/288b2d68-a082-4bb4-96a3-7a353f662aa2/concepts/4feddc57-8a5f-48a8-82f0-b3d90b8f47aa")


start headless temp chrome driver. get https://learn.udacity.com/nanodegrees/nd880/parts/cd0569/lessons/288b2d68-a082-4bb4-96a3-7a353f662aa2/concepts/4feddc57-8a5f-48a8-82f0-b3d90b8f47aa
save screenshot to main.png
quit headless temp chrome driver
